In [2]:
!pip install fake_useragent
!pip install selenium

   ---------------------------------------- 9.5/9.5 MB 16.9 MB/s eta 0:00:00
   --------------------------------------- 467.2/467.2 kB 14.7 MB/s eta 0:00:00
   ---------------------------------------- 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0


In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
# from tqdm.notebook import tqdm
from threading import Thread
from tqdm import tqdm
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# pip install chromedriver-py

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.headless = True

card_links = []

for i in tqdm(range(1, 100)):
    with webdriver.Chrome(options=options) as driver:
        driver.get(f'https://luding.ru/collection/wine/?PAGEN_1={i}')
        tree = BeautifulSoup(driver.page_source, 'html.parser')
        card_links.extend(re.findall(r'class="link-absolute" href="(\S*)"', str(tree)))
        time.sleep(0.5)

card_links = set(card_links)
card_links

In [ ]:
with open('urls.txt', 'w') as f:
    for card in card_links:
        f.write('https://luding.ru' + card)
        f.write('\n')

In [20]:
with open('bad_urls.txt') as f:
    urls = f.readlines()

In [ ]:
for url in urls[:5]:
    print(url)

https://luding.ru/collection/item/gravitation-pinot-noir-saperavi-kuban-novorossiysk-chateau-pinot--284957/

https://luding.ru/collection/item/altra-terra-kagor-bogarve-1915--383319/

https://luding.ru/collection/item/temryukskiy-pogreb-muskat--20432/

https://luding.ru/collection/item/ya-hochu-odin-plyasat-rose-fler-burlyuk--386734/

https://luding.ru/collection/item/5-fincas-emporda-do-reserva-castillo-perelada-gift-box--27139/



In [10]:
len(urls)

2376

In [3]:
cols = ['Объем', 'Бренд', 'Цена со скидкой',
        'Цена', 'Тип вина', 'Цвет', 'Сахар',
        'Страна производителя', 'Регион',
        'Аппелласьон', 'Производитель',
        'Категория', 'Бренд / Серия',
        'Сорт винограда', 'Выдержка в ёмкости',
        'Содержание алкоголя',
        'Количество в упаковке', 'Температура подачи', 
        'Friendly', 'BiO', 'Винтаж', 'Субрегион', 
        'Выдержка', 'Подарочная упаковка']

len(cols)

24

In [ ]:
from csv import DictWriter

cols = ['Объем', 'Бренд', 'Цена со скидкой',
        'Цена', 'Тип вина', 'Цвет', 'Сахар',
        'Страна производителя', 'Регион',
        'Аппелласьон', 'Производитель',
        'Категория', 'Бренд / Серия',
        'Сорт винограда', 'Выдержка в ёмкости',
        'Содержание алкоголя',
        'Количество в упаковке', 'Температура подачи', 
        'Friendly', 'BiO', 'Винтаж', 'Субрегион', 
        'Выдержка', 'Подарочная упаковка']

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--remote-debugging-pipe')
options.add_argument("--dns-prefetch-disable");
options.add_argument("--disable-gpu");
options.headless = True

driver = webdriver.Chrome(options=options)

for url in tqdm(urls):
    try:
        time.sleep(np.random.rand())
        driver.get(url)
        tree = BeautifulSoup(driver.page_source, 'html.parser')
            
        description = dict.fromkeys(cols)
        parent_div = tree.find('div', class_='rdd-body__col-description')
        description['Цена'] = tree.find('span', class_='rdd-price__main').get_text(strip=True)

        if tree.find('span', class_='rdd-price__discount'):
            description['Цена со скидкой'] = tree.find('span', class_='rdd-price__discount').get_text(strip=True)
        else:
            description['Цена со скидкой'] = description['Цена']

        description['Объем'] = parent_div.find('a', class_='rdd-volumes__link').get_text(strip=True)
        description['Бренд'] = parent_div.find('a', class_='rdd-brands__link').get_text(strip=True)

        properties = parent_div.find('div', class_='rdd-properties')
        rows = properties.findAll('div', class_='rdd-properties__group')

        for row in rows:
            names = row.findAll('div', class_='rdd-properties__name-dots')
            values = row.findAll('div', class_='rdd-properties__value')

            for name, value in zip(names, values):
                description[name.get_text(strip=True)] = value.get_text(strip=True)

        with open('data_save.csv', 'a',  encoding="utf-8") as f:
            dictwriter_object = DictWriter(f, fieldnames=cols)
            dictwriter_object.writerow(description)

#         df = df._append(description, ignore_index=True)
    except Exception as e:
        print(e)
        with open('bad_urls.txt', 'a') as f:
            f.write(url)

# df.to_csv('data.csv', index=False)
# df.head()

In [11]:
with open('data_save.csv', 'r', encoding="utf-8") as f:
    print(f.readlines())

['0.75,Chateau Pinot,580₽,725₽,Тихое,Красное,Сухое,Россия,Краснодарский край,Кубань. Новороссийск ЗГУ,Chateau Pinot,ЗГУ,Chateau PinotГравитация,Пино нуарСаперави,Стальной чан,13%,6,16-18\n', '\n']


In [9]:
description

{'Цена': '725₽',
 'Цена со скидкой': '580₽',
 'Объем': '0.75',
 'Бренд': 'Chateau Pinot',
 'Тип вина': 'Тихое',
 'Цвет': 'Красное',
 'Сахар': 'Сухое',
 'Страна производителя': 'Россия',
 'Регион': 'Краснодарский край',
 'Аппелласьон': 'Кубань. Новороссийск ЗГУ',
 'Производитель': 'Chateau Pinot',
 'Категория': 'ЗГУ',
 'Бренд / Серия': 'Chateau PinotГравитация',
 'Сорт винограда': 'Пино нуарСаперави',
 'Выдержка в ёмкости': 'Стальной чан',
 'Содержание алкоголя': '13%',
 'Количество в упаковке': '6',
 'Температура подачи': '16-18'}

In [ ]:
df.to_csv('data_save.csv', index=False)

In [6]:
df = pd.read_csv('data_save.csv', on_bad_lines='skip')
df.head()

,Объем,Бренд,Цена со скидкой,Цена,Тип вина,Цвет,Сахар,Страна производителя,Регион,Аппелласьон,...,Выдержка в ёмкости,Содержание алкоголя,Количество в упаковке,Температура подачи,Friendly,BiO,Винтаж,Субрегион,Выдержка,Подарочная упаковка
0,0.75,Chateau Pinot,580₽,725₽,Тихое,Красное,Сухое,Россия,Краснодарский край,Кубань. Новороссийск ЗГУ,...,Стальной чан,13%,6,16-18,NaN,NaN,NaN,NaN,NaN,NaN
1,0.75,Bogarve 1915,1 160₽,1 450₽,Тихое,Красное,Сладкое,Испания,NaN,NaN,...,Бочка,15%,6,14-16,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7,Olymp Winery,470₽,470₽,Тихое,Белое,Полусладкое,Россия,Краснодарский край,NaN,...,NaN,10% - 12%,6,10-12,NaN,NaN,NaN,NaN,NaN,NaN
3,0.75,Бурлюк,337₽,421₽,Тихое,Розовое,Полусухое,Россия,Крым,NaN,...,Стальной чан,10.00% - 11%,6,8-10,NaN,NaN,NaN,NaN,NaN,NaN
4,0.75,Felix Solis Avantis,954₽,1 193₽,Тихое,Розовое,Полусухое,Испания,NaN,NaN,...,Стальной чан,13%,6,8-10,NaN,NaN,NaN,NaN,NaN,NaN
